In [19]:
import pandas
import pathlib
import pydash
import requests

def value_extract(row, col):

    ''' Extract dictionary values. '''

    return pydash.get(row[col], 'value')    
    
def sparql_query(query, service):

    ''' Send sparql request, and formulate results into a dataframe. '''

    r = requests.get(service, params = {'format': 'json', 'query': query})
    data = pydash.get(r.json(), 'results.bindings')
    data = pandas.DataFrame.from_dict(data)
    for x in data.columns:    
        data[x] = data.apply(value_extract, col=x, axis=1)
    return data

def duration_conversion(row):

    ''' Convert MM:SS wikipedia runtime into seconds. '''

    seconds = str(row['Run time']).split(':')
    return (int(seconds[0])*60)+int(seconds[1])

In [3]:
# wikidata data for episodes with wikipedia links.

data_a = sparql_query("""
    SELECT DISTINCT ?story ?storyLabel ?episode ?episodeLabel ?numb ?partnumb ?article ?code
    WHERE {
        ?story wdt:P31 wd:Q28225717.
        ?story wdt:P527 ?episode.
        ?episode p:P179 ?state.
        ?state pq:P1545 ?numb. 
        ?episode p:P361 ?part.
        ?part pq:P1545 ?partnumb.
        OPTIONAL {
            ?story wdt:P2364 ?code
        }
        OPTIONAL {
            ?article schema:about ?story.
            ?article schema:inLanguage "en".
            ?article schema:isPartOf <https://en.wikipedia.org/>
        }
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }""", 'https://query.wikidata.org/sparql')

data_a.loc[data_a.story.isin(['http://www.wikidata.org/entity/Q6314527']), 'code'] = '7Cb'
data_a['numb'] = data_a['numb'].astype('int')
data_a['hinge'] = data_a['code']+'_'+data_a['partnumb']
data_a = data_a.sort_values(by=['numb'])

print(len(data_a))
data_a.head()

694


,story,article,episode,numb,partnumb,code,storyLabel,episodeLabel,hinge
195,http://www.wikidata.org/entity/Q1768718,https://en.wikipedia.org/wiki/An_Unearthly_Child,http://www.wikidata.org/entity/Q22237237,1,1,A,An Unearthly Child,An Unearthly Child,A_1
256,http://www.wikidata.org/entity/Q1768718,https://en.wikipedia.org/wiki/An_Unearthly_Child,http://www.wikidata.org/entity/Q22237238,2,2,A,An Unearthly Child,The Cave of Skulls,A_2
166,http://www.wikidata.org/entity/Q1768718,https://en.wikipedia.org/wiki/An_Unearthly_Child,http://www.wikidata.org/entity/Q22237240,3,3,A,An Unearthly Child,The Forest of Fear,A_3
220,http://www.wikidata.org/entity/Q1768718,https://en.wikipedia.org/wiki/An_Unearthly_Child,http://www.wikidata.org/entity/Q22237241,4,4,A,An Unearthly Child,The Firemaker,A_4
201,http://www.wikidata.org/entity/Q1768716,https://en.wikipedia.org/wiki/The_Daleks,http://www.wikidata.org/entity/Q22237242,5,1,B,The Daleks,The Dead Planet,B_1
260,http://www.wikidata.org/entity/Q1768716,https://en.wikipedia.org/wiki/The_Daleks,http://www.wikidata.org/entity/Q22237243,6,2,B,The Daleks,The Survivors,B_2
175,http://www.wikidata.org/entity/Q1768716,https://en.wikipedia.org/wiki/The_Daleks,http://www.wikidata.org/entity/Q22237244,7,3,B,The Daleks,The Escape,B_3
218,http://www.wikidata.org/entity/Q1768716,https://en.wikipedia.org/wiki/The_Daleks,http://www.wikidata.org/entity/Q22237245,8,4,B,The Daleks,The Ambush,B_4
253,http://www.wikidata.org/entity/Q1768716,https://en.wikipedia.org/wiki/The_Daleks,http://www.wikidata.org/entity/Q22237246,9,5,B,The Daleks,The Expedition,B_5
250,http://www.wikidata.org/entity/Q1768716,https://en.wikipedia.org/wiki/The_Daleks,http://www.wikidata.org/entity/Q22237247,10,6,B,The Daleks,The Ordeal,B_6


In [4]:
# wikipedia episode tables for stories.

spine = data_a.copy()
spine = spine[['story', 'article', 'code']].drop_duplicates()

data_b = pandas.DataFrame()
for x in range(len(spine)):
    row = spine.copy()
    row = row.iloc[x]

    grid = pandas.read_html(row['article'])
    grid = [g for g in grid if 'viewers' in str(g)]

    if len(grid) != 1:
        raise Exception('Episode grid not found.') 

    episodes = grid[0]
    for z in episodes.columns.values:
        episodes = episodes.rename(columns={z: z.split('[')[0].strip()})
    episodes['STORY'] = row['story']
    episodes['code'] = row['code']
    data_b = pandas.concat([data_b, episodes])

data_b['hinge'] = data_b['code']+'_'+data_b['Episode'].astype('str')

print(len(data_b))
data_b.head()

694


,Episode,Title,Run time,Original air date,UK viewers(millions),Appreciation Index,STORY,code,Archive,hinge
0,1,"""An Unearthly Child""",23:24,23 November 1963,4.4,63.0,http://www.wikidata.org/entity/Q1768718,A,NaN,A_1
1,2,"""The Cave of Skulls""",24:26,30 November 1963,5.9,59.0,http://www.wikidata.org/entity/Q1768718,A,NaN,A_2
2,3,"""The Forest of Fear""",23:38,7 December 1963,6.9,56.0,http://www.wikidata.org/entity/Q1768718,A,NaN,A_3
3,4,"""The Firemaker""",24:22,14 December 1963,6.4,55.0,http://www.wikidata.org/entity/Q1768718,A,NaN,A_4
0,1,"""The Dead Planet""",24:22,21 December 1963,6.9,59.0,http://www.wikidata.org/entity/Q1768716,B,NaN,B_1
1,2,"""The Survivors""",24:27,28 December 1963,6.4,58.0,http://www.wikidata.org/entity/Q1768716,B,NaN,B_2
2,3,"""The Escape""",25:10,4 January 1964,8.9,63.0,http://www.wikidata.org/entity/Q1768716,B,NaN,B_3
3,4,"""The Ambush""",24:37,11 January 1964,9.9,63.0,http://www.wikidata.org/entity/Q1768716,B,NaN,B_4
4,5,"""The Expedition""",24:31,18 January 1964,9.9,63.0,http://www.wikidata.org/entity/Q1768716,B,NaN,B_5
5,6,"""The Ordeal""",26:14,25 January 1964,10.4,63.0,http://www.wikidata.org/entity/Q1768716,B,NaN,B_6


In [23]:
# generate formatted quickstatements for episode duration.

merged_data = pandas.merge(data_a, data_b, on='hinge', how='inner')
runtime = merged_data.copy()
runtime = runtime[['episode', 'Run time']]
runtime['seconds'] = runtime.apply(duration_conversion, axis=1)
runtime['episode'] = runtime['episode'].str.split('/').str[-1]
runtime = runtime[['episode', 'seconds']]

with open(pathlib.Path.cwd() / 'quickstatements_duration.txt', 'w') as qstate1:
    for i in range(len(runtime)):
        row = runtime.iloc[i]
        quickstate_string = f"{row['episode']}|P2047|{row['seconds']}U11574|S143|Q328\n"
        qstate1.write(quickstate_string)

runtime = runtime[['episode', 'seconds']]
print(len(runtime))
runtime.head()

694


,episode,seconds
0,Q22237237,1404
1,Q22237238,1466
2,Q22237240,1418
3,Q22237241,1462
4,Q22237242,1462


In [42]:
# generate formatted quickstatements for publication date.

publication = merged_data.copy()
publication = publication.rename(columns={'Original air date': 'publication'})
publication['publication'] = pandas.to_datetime(publication['publication'])
publication = publication[['episode', 'publication']]
publication['episode'] = publication['episode'].str.split('/').str[-1]

with open(pathlib.Path.cwd() / 'quickstatements_publication.txt', 'w') as qstate2:
    for i in range(len(publication)):
        row = publication.iloc[i]
        quickstate_string = f"{row['episode']}|P577|+{str(row['publication']).replace(' ','T')}Z/11|S143|Q328\n"
        qstate2.write(quickstate_string)

print(len(publication))
publication.head()

694


,episode,publication
0,Q22237237,1963-11-23
1,Q22237238,1963-11-30
2,Q22237240,1963-12-07
3,Q22237241,1963-12-14
4,Q22237242,1963-12-21
